<a href="https://colab.research.google.com/github/phan-le-phu/CS114.K21/blob/master/%5BCase_Study%5D_Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thành viên nhóm:
- Phan Lê Phú (mssv: 18521247) 
- Trịnh Việt Hoàng (mssv: 17520522)
- Nguyễn Đức Hoan (mssv: 17520501)

Bài làm dưới đây của nhóm em dựa trên tài liệu tham khảo:
https://towardsdatascience.com/sarcasm-detection-step-towards-sentiment-analysis-84cb013bb6db



# **Mô tả vấn đề**

Trong những nghiên cứu trước đây hầu như đều sử dụng Twitter Dataset thu thập 
được nhờ quan sát hastag được gắn liền với tweet. Nhưng những dữ liệu như vậy 
thì xuất hiện sai sót khi dán nhãn hoặc gặp vấn đề về khác biệt ngôn ngữ. Hơn nữa những dòng tweet có thể reply những dòng tweet khác do đó chúng ta phải phải có cái nhìn toàn thể của ngữ cảnh mới có thể gán nhán chính xác cho tweet.

Để vượt qua những hạn chế liên quan tới độ sai lệch của Twitter datasets, chúng tôi đề xuất sử dụng News Headlines dataset cho bài toán Sarcasm Detection đã được thu thập từ hai trang web. Trang wed TheOnion mục đích đăng tải các tin tức chấm biếm của các sự kiện hiện tại và chúng tôi đã thu thập tất cả headlines từ chuyên mục News in Brief và News in Photos(những headline châm biếm). Bên cạnh đó chúng tôi thu thập các headlines (những headline không châm biếm) từ các tin tức 'thời sự' trên trang HuffPost

Sau đây là những điểm vượt trội của news dataset đối vơi Twitter datasets:
- Vì những tin tức được viết bởi những người  viết tin chuyên nghiệp với một cách thức thống nhất do đó không có lối chính tả và văn phong chuẩn mực. Điều này làm giảm 'độ thưa thớt' và tăng cơ hội tìm được pre-trained embeddings.
-Hơn nữa, mục đích chính của trang tin TheOnion là đăng tải những tin châm biếm
vì thế chúng ta nhận được độ chính xác cao của các nhãn tướng ứng với headlines hơn khi so sánh với Twitter datasets.
- Không như tweets có thể reply lại tweets khác, the news headlines chúng ta thu được thì chỉ có ý nghĩa đối với bài báo mà nó được xuất hiện cùng. Điều này giúp chúng ta nhìn nhận được các yếu tố châm biếm thực sự.





## **Mô tả dataset**

Dataset bao gồm ba cột:
*   ```article_link``` (type: Object): chứa links dẫn đến news articles
*   ```headline``` (type: Object): chứa headlines của news articles
*   ```is_sarcastic``` (type: int64): giá trị 0 nếu thu thập từ trang HuffPost(không phải sarcastic) hoặc giá trị 1 nếu thu thập từ trang TheOnion(sarcastic)

## **Source Code**

Đầu tiên import các moudle cần thiết để xử lý bài toán.
- moudle pandas cần cho việc đọc file json 
- moudle numpy
- moudle regex
- hàm PorterStemmer từ moulde nltk.stem.porter dùng để stem các từ trong câu


In [1]:
import pandas as pd
import numpy as np, re, time
from nltk.stem.porter import PorterStemmer

- Hàm train_test_split được import để chia dataset thành tập dữ liệu train và tập dữ liệu test
- Vì đây là bài toán classification nên em import các model cài đặt thuật toán sử dụng được cho bài toán này. Mục tiêu là model nào cho kết quả thu được tốt nhất em sẽ chọn model đó để đánh giá với tập dữ liệu tự thu thập.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

- tải tập dataset đã tải từ kaggle lên colab


In [3]:
from google.colab import files
uploaded = files.upload()

Saving Sarcasm_Headlines_Dataset_v2.json to Sarcasm_Headlines_Dataset_v2.json


- đọc file chứa dataset được tải lên bên trên bằng hàm read_json của module pandas thu được một DataFrame và dùng biến data để quản lý nó.

In [4]:
data = pd.read_json('Sarcasm_Headlines_Dataset_v2.json', lines = True)

- Hiển thị bố cục của Dataframe được quản lý bởi biến data. Dễ thấy nó bao gồm ba cột(is_scarastic, headline và article_link) mỗi cột chứa các thông tin như đã được miêu tả trong phần mô tả dataset.


In [5]:
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


- dataset bao gồm 28619 examples

In [6]:
data.shape

(28619, 3)


- trong 28619 examples không có giá Null

In [7]:
print(data.isnull().any(axis = 0))

is_sarcastic    False
headline        False
article_link    False
dtype: bool


## **Cleaning the data**

Cột headline có chứa một vài ký tự đặc biệt chúng ta không cần xét đến  ở đây đối với từng headline em chỉ xét đến các ký tự trong tập a-z và A-Z.
- dòng code bên dưới sẽ loop qua từng phần tử trong headline column ứng với mỗi vòng lặp là một headline **s**. Chuỗi  **s** này sẽ được xử lý bằng hàm ẩn lambda. Trong hàm ẩn lambda sử dụng hàm sub của moulde re, hàm này sẽ thấy thế những ký tự match với biểu thức regex [^a-zA-Z] (những ký từ không trong tập a-z và A-Z sẽ match) bằng ký tự ' ' (1 lần space). Hàm ẩn sẽ trả về chuỗi mới sau khi được xử lý. Sau đó chuỗi này sẽ được thay thế vào index tương ứng của chuỗi **s** trong headline column

In [8]:
# Replacing special symbols and digits in headline column
# Using Regular Expresion
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

## **Feature and label extractions**

Như đã mô  tả bên trên dataset của chúng ta có 3 cột. Tuy nhiên, ```article_link``` thì không quan trọng đối với bài toán bởi vì không có thì cũng không ảnh hưởng tới mục tiêu bài toán việc nó được lưu trữ là để thuận tiện cho việc thu thập thêm dữ liệu sau này. Vì vậy chỉ có   ```headline```  là feature duy nhất. Và, ```is_sarcastic``` là label duy nhất của dataset.

In [9]:
# Getting features and labels
features = data['headline']
labels = data['is_sarcastic']

## **Stemming of features (tại sao phải làm bước này?)**

Ở bước này ta chuyển các từ đã được biến đổi thành các từ gốc của nó.

Ví dụ minh họa:

*   original words: ```reading``` and ```reader```
*   stemmed words: ```read```




- dòng  code đầu tiên dùng biến ps để quản lý đối tượng PorterStemmer
- dòng code tiếp theo sẽ loop qua từng chuỗi **x** trong dataframe(chứa cột headline) được quản lý bởi biến features. Chuỗi **x** sẽ được xử lý bằng hàm ẩn lambda. Hàm này sẽ cắt chuỗi thành các từ riêng biệt dựa vào yếu tố các từ được cách nhau bởi dấu cách và trả về kết quả là một mảng chứa các từ sau khi được cắt từ chuỗi **x**. Cuối cùng nó sẽ được lưu vào index tương ứng của chuỗi **x**
trong dataframe được quản lý bởi biến features
- dòng code cuối cùng loop qua từng mảng **x** trong dataframe(chứa cột headline) được quản lý bởi biến features. Mảng **x** sẽ được xử lý bằng hàm ẩn lambda. Hàm này sẽ loop qua từng phân tử trong mảng **x**, mỗi phần tử sẽ được xử lý bằng hàm ps.tem() đễ tìm từ gốc tương tứng của nó sau đó các phần tử sẽ được kết lại thành một mảng trở thành tham số trong hàm join, hàm join sẽ hợp các phần tử trong mảng vừa nhận thành một chuỗi và phân chia chúng bằng dấu cách, kết quả là ta thu được một chuỗi. Cuối cùng nó sẽ được lưu vào index tương ứng của mảng **x** trong dataframe được quản lý bởi biến features

In [10]:
# Stemming the data
ps = PorterStemmer()

features = features.apply(lambda x: x.split())
features = features.apply(lambda x: ' '.join([ps.stem(word) for word in x]))



## **Vectorization of features**

Vì chúng ta không thể feed model bằng text do đó em sử dụng kỹ thuật sau để biểu diễn một đoạn text được nhận thành một vector 1 chiều có số features max là 5000 mỗi feature trong vector là chỉ số  TF-IDF của một từ bên trong đoạn text: ***TF-IDF (Term Frequency-Inverse Document Frequency)***.


***TF - Term Frequency***: Xem xét tuần suất xuất hiện của một từ trong text. Có giá trị nằm trong khoảng  [0-1]. Nếu một từ trong text xuất hiện càng nhiều lần thì giá trị càng tiến về 1 và ngược lại

***IDF - Inverse Document Frequency***: Xem xét tuần xuất xuất hiện của một từ trong corpus. Nều từ đó xuất hiện càng ít thì giá trị càng lớn và ngược lại.
[Link tham khảo](https://vi.wikipedia.org/wiki/Tf%E2%80%93idf)










*   import class TfidfVectorizer từ moulde sklearn và dùng biến tv để quản lý nó
*   cài đặt max_feaures  cho data bằng 5000
*   convert dataframe thành list
*   sử dụng hàm fit_tranform  với tham số là một list chứa các headlines. Sau  khi thực hiện nó sẽ trả về một csr.matrix. Khi dùng hàm print  với tham số là nó, thì sẽ hiện thị nhiều dòng. Mỗi dòng sẽ có cấu trúc '(a, b)    data'. Trong đó a là index của text trong list features, b là index của từ đó trong tập set(tập set là một tập chứa các từ vựng được tìm thấy trong list features và được sắp xếp theo bảng chữ cái), data là chỉ số if-idf tương ứng.
*    hàm toarray sẽ chuyển csr.matrix bên trên thành một ndarray chứ một tập các vector biểu diễn ý nghĩa bằng số cho text.



In [11]:
# Vectorizing the data with maximum of 5000 features
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
headlines = tv.fit_transform(features).toarray()

## **Training and Testing data**

* dùng hàm train_test_split với các tham số là features( cột headlines), labels(cột is_sarcastic), test_size=0.05 (5% cho dữ liệu test), random_state=0 và dùng các biến tương ứng với tên của nó bên dưỡi để quản lý.

In [12]:
# Getting training and testing dataset
features_train, features_test, labels_train, labels_test = train_test_split(headlines, labels, test_size = .05, random_state = 0)

## **Training and Testing of models**

Train các loại model khác nhau bằng cách sử dụng nhiều thuật toán máy học.
1.   Linear Support Vector Classifier
2.   Gaussian Naive Bayes
3.   Logistic Regression
4.   Random Forest Classifier




In [13]:
# Model 1: Linear Support Vector Classifier

lsvc = LinearSVC()

# Training the model
lsvc.fit(features_train, labels_train)

# Getting the score of train and test data
print(lsvc.score(features_train, labels_train))
print(lsvc.score(features_test, labels_test))

0.9069074591731646
0.8322851153039832


In [14]:
# Model 2: Gaussian Naive Bayes

gnb = GaussianNB()

# Training the model
gnb.fit(features_train, labels_train)

# Getting the score of train and test data
print(gnb.score(features_train, labels_train))
print(gnb.score(features_test, labels_test))

0.7977416507282624
0.7169811320754716


In [15]:
# Model 3: Logistic Regression

lr = LogisticRegression()

# Training the model
lr.fit(features_train, labels_train)

# Getting the score of train and test data
print(lr.score(features_train, labels_train))
print(lr.score(features_test, labels_test))

0.8778137413564808
0.8252969951083159


In [16]:
# Model 4: Random Forest Classifier

rfc = RandomForestClassifier(n_estimators= 10, random_state= 0)

# Training the model
rfc.fit(features_train, labels_train)

# Getting the score of train and test data
print(rfc.score(features_train, labels_train))
print(rfc.score(features_test, labels_test))

0.9883404443136677
0.7721872816212438


Nhận thấy model sử dụng thuật toán linear support vector classifier cho điểm score cao trên cả train dataset và test dataset khi so với các model sử dụng các thuật toán khác. Model số 4 sử dụng thuật toán random forest classifer mặc dù cho điểm score rất cao với train dataset những lại khá là thập trong tập test dataset đây có thể là dấu hiêu của model đã bị overfit. Do đó em chọn model sử dụng thuật toán support vector classifier làm model để thực hiện các bước tiếp theo.

## **Ứng dụng model**


- Yêu cầu người dùng nhập headline cần detect
- clean headline và stem word  cho headline tương tự với các bác đã thực hiên cho dataset
- thêm headline và list các text của dataset được quản lý bới biến features sau đó biểu diễn  ý nghĩa của headline bằng 1 vecor có max 5000 features
- sau đó tiến hành predict

In [17]:
print("Nhập headline cần detect:")
headline = input()

headline = re.sub('[^a-zA-Z]', ' ', headline)

headline = ' '.join([ps.stem(word) for word in headline.split()])

features.append(headline)

tv = TfidfVectorizer(max_features=5000)
headline = tv.fit_transform(features).toarray()[-1].reshape(1,-1)




print("Đang xử lý.......")
if rfc.predict(headline)[0] == 1:
  print("Đây là headline của tin tức châm biếm!!!")
else:
    print("Đây là headline của tin tức thời sự!!!")


Nhập headline cần detect:
Nancy Pelosi Calls Jamaal Bowman To Scold Him For Winning Primary
Đang xử lý.......
Đây là headline của tin tức thời sự!!!


## **Kiểm tra kết quả của model với dataset tự thu thập được**


Đọc file csv của datasets thu thập được bằng hàm read_csv của thư viện pandas

In [18]:
onion_dataset = pd.read_csv("https://raw.githubusercontent.com/phan-le-phu/CS114.K21/master/collection_dataset_for_sarcasm_detection/onion_titles.csv")
huffpost_dataset = pd.read_csv("https://raw.githubusercontent.com/phan-le-phu/CS114.K21/master/collection_dataset_for_sarcasm_detection/huffpost_tittles.csv")

Mỗi dataframe thu được có bốn cột trong đó cột đầu tiên là cột index, cột thứ hai là cột is_sarcastic, cột thứ ba là headline và cột cuối cùng là article_link.
Trong tập dataset của onion có 1506 headline còn trong tập dataset của huffpost chứa 1008 headline.

In [19]:
print(onion_dataset.shape)
print(huffpost_dataset.shape)

(1506, 4)
(1008, 4)


In [20]:
print(huffpost_dataset)

      Unnamed: 0  ...                                       article_link
0              0  ...  https://www.huffpost.com/entry/hickenlooper-co...
1              1  ...  https://www.huffpost.com/entry/genderfluid-com...
2              2  ...  https://www.huffpost.com/entry/ibm-facial-reco...
3              3  ...  https://www.huffpost.com/entry/sophie-turner-s...
4              4  ...  https://www.huffpost.com/entry/eliot-engel-hot...
...          ...  ...                                                ...
1003        1003  ...  https://www.huffpost.com/entry/george-w-bush-g...
1004        1004  ...  https://www.huffpost.com/entry/street-art-blac...
1005        1005  ...  https://www.huffpost.com/entry/amc-theatres-re...
1006        1006  ...  https://www.huffpost.com/entry/temecula-califo...
1007        1007  ...  https://www.huffpost.com/entry/gop-sheriff-cor...

[1008 rows x 4 columns]


Chuẩn bị dataset



- liên kết hai dataset


In [21]:
new_data = onion_dataset.append(huffpost_dataset)
print(new_data)

      Unnamed: 0  ...                                       article_link
0              0  ...  https://www.theonion.com/mental-health-experts...
1              1  ...  https://www.theonion.com/the-onion-s-father-s-...
2              2  ...  https://sports.theonion.com/staples-center-emp...
3              3  ...  https://www.theonion.com/father-s-day-gifts-th...
4              4  ...  https://www.theonion.com/huh-boyfriend-s-ex-ju...
...          ...  ...                                                ...
1003        1003  ...  https://www.huffpost.com/entry/george-w-bush-g...
1004        1004  ...  https://www.huffpost.com/entry/street-art-blac...
1005        1005  ...  https://www.huffpost.com/entry/amc-theatres-re...
1006        1006  ...  https://www.huffpost.com/entry/temecula-califo...
1007        1007  ...  https://www.huffpost.com/entry/gop-sheriff-cor...

[2514 rows x 4 columns]


In [22]:
print(new_data.isnull().any(axis = 0))

Unnamed: 0      False
is_sarcastic    False
titles          False
article_link    False
dtype: bool


In [23]:
new_features = new_data["titles"]
new_labels = new_data["is_sarcastic"]

Xử lý dataset tương tự như vơi cách xử lý bên trên

In [24]:
# Replacing special symbols and digits in headline column
# Using Regular Expresion
new_features = new_features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

# Stemming the data
ps = PorterStemmer()

new_features = new_features.apply(lambda x: x.split())
new_features = new_features.apply(lambda x: ' '.join([ps.stem(word) for word in x]))

- lấy index from để xác định khoảng các text cần kiểm tra
- thêm các headline mới vào tập các headline cũ

In [25]:
from_index = len(features)

features += list(new_features)

- Biểu diễn các healine trong tập dataset mới bằng các vector

In [26]:
# Vectorizing the data with maximum of 5000 features
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
new_headlines = tv.fit_transform(features).toarray()[from_index:]

- Sử dụng model với tập dataset vừa thu thập được.

In [27]:
print(lsvc.score(new_headlines, new_labels))

0.5095465393794749


## **Nhận xét kết quả**

Kết quả thu được đối với tập dataset mới của model chỉ được 0.50597 khá là thấp so với kết quả kiểm nghiêm của model trên train dataset và test dataset. Em dự đoán một vài nguyên nhân sau.
- có thể model bị overfit với train dataset do kêt quả thu được của model khá cao là 0.90 nhưng test dataset thu được thì thấp hơn là  0.83.
- dữ liệu huấn luyến chưa đủ nhiều để train model
- bước features engneering chữa biểu diễn được chính xác ý nghĩa của câu bằng số.